In [ ]:
import sklearn
from sklearn.linear_model import Ridge
from scipy.io import loadmat
import numpy as np
from matplotlib import pyplot as plt
import functionList

In [ ]:
raw = loadmat(r"C:\Users\parkf\Desktop\summer2021\eegData\for_Barry\282L.mat") #change this to try different one off files
X = raw['data']  
mask = raw['mask']
l = X.shape[0]
c = X.shape[1]

In [ ]:
memVec = [4, 8, 12]
ridgeVec = [.1, .5, 1]
showChannels = 1 #toggel between 0 and 1 for channel by channel info

cvNum = 3

s = (len(memVec), len(ridgeVec), c, cvNum)
error = np.zeros(s)
for memNumber in memVec:
    featureMat = functionList.buildFeatureMat(X,memNumber)
    xClean = functionList.removeTrash(X,mask,memNumber)
    featureMat = functionList.removeTrash(featureMat,mask,memNumber)
    for regularizer in ridgeVec: 
        crossVal = sklearn.model_selection.KFold(n_splits=cvNum)
        for thisChannel in range(c):
            adjustedReg = regularizer*np.linalg.norm(xClean[:,thisChannel])
            cvRun = 0
            for train, test in crossVal.split(featureMat):
                pred = functionList.predRrOutput(featureMat[train,:], featureMat[test,:], xClean[train,thisChannel], adjustedReg, 0)
                err = functionList.findMSE(pred, xClean[test,thisChannel])
                error[memVec.index(memNumber), ridgeVec.index(regularizer), thisChannel, cvRun] = err
                cvRun = cvRun + 1
errNoCv = np.mean(error, 3)
errNoCh = np.mean(errNoCv, 2)

In [ ]:
if showChannels == 1:
    for ch in range(c):
        titleString = f"Memory vs MSE, channel {ch}"
        plt.title(titleString)
        for i in range(len(ridgeVec)):
            plt.plot(memVec[:], errNoCv[:, i, ch], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
        plt.legend()
        plt.show()

titleString = f"Memory vs MSE, for subject, channel's averaged"
plt.title(titleString)
for i in range(len(ridgeVec)):
    plt.plot(memVec[:], errNoCh[:, i], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
    plt.xlabel("Number of memory points")
    plt.ylabel("MSE")
plt.legend()
plt.show()
